## Part 2

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

C:\Users\Mohamed\anaconda3\envs\pythonProject1\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Mohamed\anaconda3\envs\pythonProject1\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\Mohamed\anaconda3\envs\pythonProject1\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
wikiURL = "http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(wikiURL)
soup = BeautifulSoup(page.content, 'html.parser')

table_contents=[]
table=soup.find("table")

for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

#print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df.columns = ['Postal Code', 'Borough', 'Neighborhood']

In [3]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
neighborhoods = pd.merge(df,coordinates,on='Postal Code', how='inner')
neighborhoods.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [6]:
colors = {'Scarborough':'red', 'North York':'blue', 'East York':'green',
         'East Toronto':'darkgreen', 'East York/East Toronto':'white', 'Central Toronto':'brown',
         'Downtown Toronto':'grey', 'Downtown Toronto Stn A':'purple', 'York':'yellow',
         'West Toronto':'darkblue', "Queen's Park":'lightblue', 'Mississauga':'lightred',
         'East Toronto Business':'darkred', 'Etobicoke':'black', 'Etobicoke Northwest':'orange'}

print('The dataframe has {} boroughs.'.format(
        len(neighborhoods['Borough'].unique()))
)

The dataframe has 15 boroughs.


In [9]:
import folium
map_toronto = folium.Map(location=[neighborhoods.Latitude.mean(),
                                   neighborhoods.Longitude.mean()], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'],
                                           neighborhoods['Longitude'],
                                           neighborhoods['Borough'],
                                           neighborhoods['Neighborhood']):
    
    label = '{} - {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius='4',
        popup=label,
        color=colors[borough],
        fill=True,
        fill_color='yellow',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [10]:
from folium import plugins
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[neighborhoods.Latitude.mean(),
                                   neighborhoods.Longitude.mean()], zoom_start=10)

incidents = plugins.MarkerCluster().add_to(map_toronto)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'],
                                           neighborhoods['Longitude'],
                                           neighborhoods['Borough'],
                                           neighborhoods['Neighborhood']):
    
    label = '{} - {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius='4',
        popup=label,
        color=colors[borough],
        fill=True,
        fill_color='yellow',
        fill_opacity=0.7,
        parse_html=False).add_to(incidents)  
    
map_toronto